In [4]:
df = spark.sql("SELECT * FROM bing_lake_db.tbl_latest_news")
display(df)

StatementMeta(, 9bb8f50d-259a-4091-9ea1-9f952c0202c6, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 4d5adde9-6e45-4bdc-89ca-6f8d32ce75e0)

In [5]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, 9bb8f50d-259a-4091-9ea1-9f952c0202c6, 7, Finished, Available)

In [6]:
# Import the model and configure the input and output columns
model = (AnalyzeText()
        .setTextCol("description")
        .setKind("SentimentAnalysis")
        .setOutputCol("response")
        .setErrorCol("error"))

StatementMeta(, 9bb8f50d-259a-4091-9ea1-9f952c0202c6, 8, Finished, Available)

In [7]:
#Apply the model to our dataframe
result = model.transform(df)   

StatementMeta(, 9bb8f50d-259a-4091-9ea1-9f952c0202c6, 9, Finished, Available)

In [8]:
display(result)

StatementMeta(, 9bb8f50d-259a-4091-9ea1-9f952c0202c6, 10, Finished, Available)

SynapseWidget(Synapse.DataFrame, 126d5b5c-d778-480c-b074-04d5395cc2d1)

In [9]:
#Create Sentiment Column
from pyspark.sql.functions import col

sentiment_df = result.withColumn("Sentiment",col("response.documents.sentiment"))

StatementMeta(, 9bb8f50d-259a-4091-9ea1-9f952c0202c6, 11, Finished, Available)

In [10]:
display(sentiment_df)

StatementMeta(, 9bb8f50d-259a-4091-9ea1-9f952c0202c6, 12, Finished, Available)

SynapseWidget(Synapse.DataFrame, 94edc94e-4b6a-4506-82ec-12ee7180c6fa)

In [11]:
sentiment_df_final = sentiment_df.drop("error","response")

StatementMeta(, 9bb8f50d-259a-4091-9ea1-9f952c0202c6, 13, Finished, Available)

In [12]:
display(sentiment_df_final)

StatementMeta(, 9bb8f50d-259a-4091-9ea1-9f952c0202c6, 14, Finished, Available)

SynapseWidget(Synapse.DataFrame, a2c4384d-2d62-41e1-a629-b886049e165d)

In [13]:
from pyspark.sql.functions import col, to_date

sentiment_df_final = sentiment_df_final.withColumn("datePublished", to_date(col("datePublished"), "dd-MMM-yyyy"))

StatementMeta(, 9bb8f50d-259a-4091-9ea1-9f952c0202c6, 15, Finished, Available)

In [14]:
display(sentiment_df_final)

StatementMeta(, 9bb8f50d-259a-4091-9ea1-9f952c0202c6, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, cc7b1175-15a6-4a4f-936a-310bb6912475)

# Type 1 Merge

In [15]:
from pyspark.sql.utils import AnalysisException

try:

    table_name = 'bing_lake_db.tbl_sentiment_analysis'
    sentiment_df_final.write.format('delta').saveAsTable(table_name)

except AnalysisException:
    print("Table Already Exists")
    sentiment_df_final.createOrReplaceTempView("vw_sentiment_df_final")
    spark.sql(f"""MERGE INTO {table_name} AS target_table
                  USING vw_sentiment_df_final AS source_view
                  ON source_view.url = target_table.url
                  WHEN MATCHED AND (
                      source_view.title <> target_table.title OR 
                      source_view.description <> target_table.description OR 
                      source_view.category <> target_table.category OR 
                      source_view.image <> target_table.image OR 
                      source_view.provider <> target_table.provider OR 
                      source_view.datePublished <> target_table.datePublished
                  )
                  THEN UPDATE SET *
                  WHEN NOT MATCHED THEN INSERT *
                """)

StatementMeta(, 9bb8f50d-259a-4091-9ea1-9f952c0202c6, 17, Finished, Available)

Table Already Exists


In [16]:
df1 = spark.sql("SELECT * FROM bing_lake_db.tbl_sentiment_analysis")
display(df1)

StatementMeta(, 9bb8f50d-259a-4091-9ea1-9f952c0202c6, 18, Finished, Available)

SynapseWidget(Synapse.DataFrame, 10e62901-5402-4a9d-81df-92fd9bf9116e)